In [ ]:
import pyvisa
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

class AWG_controller:
    def __init__(self, ax = None, canvas = None):
        self.rm = pyvisa.ResourceManager()
        self.resource = None
        self.ax = ax
        self.canvas = canvas

        # Device limits (user cannot change)
        self.device_voltage_limit = 30.0  # volts
        self.device_power_limit = self.device_voltage_limit ** 2 / 50  # assume 50Ω load
        self.device_freq_limit = 6e9  # 6 GHz

        # Software limits (user-defined but must stay <= device limits)
        self.software_voltage_limit = self.device_voltage_limit
        self.software_power_limit = self.device_power_limit
        self.software_freq_limit = self.device_freq_limit

        # User parameters
        self.center_freq = None
        self.bandwidth = None
        self.freq_step = None
        self.start_power = None
        self.stop_power = None
        self.power_step = None

    def power_on(self):
        if self.resource:
            self.resource.write("OUTP ON")
            print("AWG Output ON")

    def power_off(self):
        if self.resource:
            self.resource.write("OUTP OFF")
            print("AWG Output OFF")

    def connect(self, ip_address):
        try:
            self.resource = self.rm.open_resource(f"TCPIP0::{ip_address}::inst0::INSTR")
            self.resource.write("*IDN?")
            print("Connected to device:", self.resource.read().strip())
        except Exception as e:
            print("Connection error:", e)

    def set_software_voltage_limit(self, voltage):
        if voltage <= self.device_voltage_limit:
            self.software_voltage_limit = voltage
            print(f"Software voltage limit set to {voltage} V")
        else:
            print("Error: Voltage exceeds device limit")

    def set_software_power_limit(self, power):
        if power <= self.device_power_limit:
            self.software_power_limit = power
            print(f"Software power limit set to {power} W")
        else:
            print("Error: Power exceeds device limit")

    def set_software_freq_limit(self, freq):
        if freq <= self.device_freq_limit:
            self.software_freq_limit = freq
            print(f"Software frequency limit set to {freq / 1e6} MHz")
        else:
            print("Error: Frequency exceeds device limit")

    def set_user_parameters(self, center_freq, bandwidth, freq_step, start_power, stop_power, power_step):

        self.center_freq = center_freq
        self.bandwidth = bandwidth
        self.freq_step = freq_step
        self.start_power = start_power
        self.stop_power = stop_power
        self.power_step = power_step
        
        start_f = center_freq - bandwidth / 2
        stop_f = center_freq + bandwidth / 2

        if stop_f > self.software_freq_limit:
            print("Error: Frequency range exceeds software/device limit")
            return False
        if stop_power > self.software_power_limit:
            print("Error: Power exceeds software/device limit")
            return False

        

        print("User parameters set successfully.")
        return True

    def sweep_center_band_power(self):
        if any(param is None for param in [self.center_freq, self.bandwidth, self.freq_step, self.start_power, self.stop_power, self.power_step]):
            print("User parameters not set.")
            return

        self.center_f = self.center_freq
        self.bandwidth = self.bandwidth
        self.f_step = self.freq_step
        self.start_p = self.start_power
        self.stop_p = self.stop_power
        self.p_step = self.power_step

        self.start_f = self.center_f - self.bandwidth / 2
        self.stop_f = self.center_f + self.bandwidth / 2
        self.sample_rate = 64e9
        self.duration = 1e-6
        t = np.arange(0, self.duration, 1 / self.sample_rate)

        while self.stop_f <= self.device_freq_limit:
            print(f"Sweeping: Freq = {self.start_f/1e6:.2f} to {self.stop_f/1e6:.2f} MHz")

            for power in np.arange(self.start_p, self.stop_p + self.p_step, self.p_step):
                print(f'Power is {power}')
                for freq in np.arange(self.start_f, self.stop_f + self.f_step, self.f_step):
                    amplitude = 10 ** (power / 20)
                    signal = amplitude * np.sin(2 * np.pi * freq * t)
                    if self.resource:
                        try:
                            self.freque = freq
                            self.v_rms = np.sqrt((10**((power - 30)/10)) * 50)
                            self.ascii_data = ",".join([f"{x:.6f}" for x in np.clip(signal, -1.0, 1.0)])
                            
                            #Call the generate waveform function
                            self.generte_standard_waveform(self.ascii_data, self.freque, self.v_rms, signal, power)
                        except Exception as e:
                            print("SCPI command error:", e)

            self.start_f += self.f_step
            self.stop_f += self.f_step
            self.center_f += self.f_step

        self.resource.write("OUTP OFF")
        
        print("Sweep complete.")

    def load_waveform_file(self, file_path, waveform_name="UserWaveform"):
        if self.resource is None:
            print("Error: AWG not connected.")
            return False

        if not os.path.isfile(file_path):
            print("Error: File not found.")
            return False

        file_ext = os.path.splitext(file_path)[1].lower()
        try:
            if file_ext == ".csv":
                print(f"Loading CSV waveform: {file_path}")
                data_array = np.loadtxt(file_path, delimiter=',', skiprows=1)
                if data_array.ndim > 1:
                    data_array = data_array[:, 1]
            elif file_ext == ".mat":
                print(f"Loading MAT waveform: {file_path}")
                mat = scipy.io.loadmat(file_path)
                waveform_key = next((k for k in mat.keys() if not k.startswith('__')), None)
                if waveform_key is None:
                    print("Error: No waveform array found in .mat file.")
                    return False
                data_array = np.array(mat[waveform_key]).flatten()
            else:
                print("Unsupported file format. Use .csv or .mat")
                return False

            data_array = np.clip(data_array, -1.0, 1.0)
            ascii_data = ",".join([f"{x:.6f}" for x in data_array])
            self.generate_waveform_from_file(ascii_data, waveform_name, data_array, file_path)
            
            return True

        except Exception as e:
            print("Waveform loading failed:", e)
            return False
    
    def generte_standard_waveform(self, ascii_data, freque, v_rms, signal, power):
        self.resource.write(f":MMEM:DATA:DEF 'SweepSignal', {len(ascii_data)}")
        self.resource.write_binary_values(":MMEM:DATA 'SweepSignal', ", ascii_data.encode('ascii'), datatype='B')

        # 1. Set function mode to ARB (Arbitrary)
        self.resource.write("FUNC:MODE ARB")

        # 2. Select the waveform (from memory) to the trace (DAC buffer)
        self.resource.write(":TRACE:DATA:DAC 'SweepSignal'")

        # 3. Set the carrier frequency (not generic FREQ)
        self.resource.write(f":CARR1:FREQ {freque}")

        # 4. Set output voltage amplitude
        self.resource.write(f":VOLT {v_rms}")

        # 5. Enable output on channel 1
        self.resource.write("OUTP1 ON")


        plt.ion()
        fig, ax = plt.subplots(figsize=(10, 4))
        line, = ax.plot([], [], lw=1.5)
        ax.set_ylim(-2, 2)
        ax.set_xlim(0, 1000)
        ax.set_title("Live Waveform Simulation During Sweep")
        ax.set_xlabel("Sample Index")
        ax.set_ylabel("Amplitude (V)")
        fig.tight_layout()

        line.set_ydata(signal[:1000])
        line.set_xdata(np.arange(1000))
        ax.set_title(f"Freq = {freque/1e6:.2f} MHz | Power = {power:.2f} dBm")
        fig.canvas.draw()
        fig.canvas.flush_events()

        plt.ioff()
        if self.ax and self.canvas:
            self.ax.clear()
            self.ax.plot(signal, color='blue')  # or signal[:1000]
            self.ax.set_title(f"Waveform: Freq = {freque/1e6:.2f} MHz | Power = {power:.2f} dBm")
            self.ax.set_xlabel("Sample Index")
            self.ax.set_ylabel("Amplitude")
            self.ax.set_ylim(-1.2, 1.2)
            self.canvas.draw()


    def generate_waveform_from_file(self, ascii_data, waveform_name, data_array, file_path):
        try:
            # Step 1: Define memory file size
            self.resource.write(f":MMEM:DATA:DEF '{waveform_name}', {len(ascii_data)}")
    
            # Step 2: Send waveform data in 488.2 block format
            self.resource.write_binary_values(f":MMEM:DATA '{waveform_name}', ", ascii_data.encode('ascii'), datatype='B')
    
            # Step 3: Set output mode to arbitrary waveform
            self.resource.write(":FUNC:MODE ARB")
    
            # Step 4: Load waveform into DAC (output buffer)
            self.resource.write(f":TRACE:DATA:DAC '{waveform_name}'")
    
            # Step 5: Set other parameters as needed (example: voltage or carrier frequency)
            # self.resource.write(":VOLT 1.0")  # Optional
            # self.resource.write(":CARR1:FREQ 1e9")  # Optional
    
            # Step 6: Enable output on channel 1
            self.resource.write("OUTP1 ON")

            plt.ion()
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.set_title(f"Waveform Preview: {os.path.basename(file_path)}")
            ax.set_xlabel("Sample Index")
            ax.set_ylabel("Amplitude")
            ax.set_xlim(0, min(1000, len(data_array)))
            ax.set_ylim(np.min(data_array[:1000]) * 1.2, np.max(data_array[:1000]) * 1.2)
            ax.plot(data_array[:1000], color='blue')
            fig.tight_layout()
            fig.canvas.draw()
            fig.canvas.flush_events()
            plt.pause(1.5)
            plt.ioff()
            if self.ax and self.canvas:
                self.ax.clear()
                self.ax.plot(data_array[:1000], color='blue')  # or signal[:1000]
                self.ax.set_title(f"Waveform:")
                self.ax.set_xlabel("Sample Index")
                self.ax.set_ylabel("Amplitude")
                self.ax.set_ylim(-1.2, 1.2)
                self.canvas.draw()

    
            print("Waveform loaded and output started.")
    
        except Exception as e:
            print("Cannot generate waveform:", e)





In [ ]:
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QLineEdit,
    QPushButton, QFileDialog, QMessageBox, QGroupBox, QFormLayout
)
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import sys
import threading

class AWG_GUI_Interface(QWidget):
    def __init__(self, awg_controller):
        super().__init__()
        self.awg = AWG_controller(ax = self.ax, canvas = self.canvas)
        self.setWindowTitle("AWG GUI Interface")
        self.setGeometry(100, 100, 1000, 600)

        # Main layout
        layout = QHBoxLayout()
        layout.addLayout(self._create_controls(), 1)
        layout.addWidget(self._create_plot_canvas(), 3)

        self.setLayout(layout)

    def _create_controls(self):
        vbox = QVBoxLayout()
        
        # Group box for connection
        conn_box = QGroupBox("Connection")
        conn_layout = QFormLayout()
        self.ip_input = QLineEdit("192.168.1.2")
        conn_button = QPushButton("Connect")
        conn_button.clicked.connect(self._connect_awg)
        conn_layout.addRow("IP Address:", self.ip_input)
        conn_layout.addRow(conn_button)
        conn_box.setLayout(conn_layout)

        # Group box for limits
        limit_box = QGroupBox("Software Limits")
        limit_layout = QFormLayout()
        self.voltage_input = QLineEdit("30")
        self.power_input = QLineEdit(str(self.awg.device_power_limit))
        self.freq_input = QLineEdit(str(self.awg.device_freq_limit))
        set_limits = QPushButton("Set Limits")
        set_limits.clicked.connect(self._set_limits)
        limit_layout.addRow("Voltage Limit (V):", self.voltage_input)
        limit_layout.addRow("Power Limit (W):", self.power_input)
        limit_layout.addRow("Freq Limit (Hz):", self.freq_input)
        limit_layout.addRow(set_limits)
        limit_box.setLayout(limit_layout)

        # Group box for sweep parameters
        param_box = QGroupBox("Generate Waveform")
        param_layout = QFormLayout()
        self.center_freq_input = QLineEdit()
        self.bandwidth_input = QLineEdit()
        self.freq_step_input = QLineEdit()
        self.start_power_input = QLineEdit()
        self.stop_power_input = QLineEdit()
        self.power_step_input = QLineEdit()
        param_layout.addRow("Center Freq (Hz):", self.center_freq_input)
        param_layout.addRow("Bandwidth (Hz):", self.bandwidth_input)
        param_layout.addRow("Freq Step (Hz):", self.freq_step_input)
        param_layout.addRow("Start Power (dBm):", self.start_power_input)
        param_layout.addRow("Stop Power (dBm):", self.stop_power_input)
        param_layout.addRow("Power Step (dB):", self.power_step_input)

        set_params_btn = QPushButton("Set Parameters")
        set_params_btn.clicked.connect(self._set_user_parameters)
        param_layout.addRow(set_params_btn)
        param_box.setLayout(param_layout)

        # Control Buttons
        power_on_btn = QPushButton("Power ON")
        power_on_btn.clicked.connect(self.awg.power_on)

        power_off_btn = QPushButton("Power OFF")
        power_off_btn.clicked.connect(self.awg.power_off)

        sweep_btn = QPushButton("Generate waveform")
        sweep_btn.clicked.connect(lambda: threading.Thread(target=self.awg.sweep_center_band_power).start())

        load_waveform_btn = QPushButton("Load Waveform File")
        load_waveform_btn.clicked.connect(self._load_waveform)

        # Add widgets to sidebar
        vbox.addWidget(conn_box)
        vbox.addWidget(limit_box)
        vbox.addWidget(param_box)
        vbox.addWidget(power_on_btn)
        vbox.addWidget(power_off_btn)
        vbox.addWidget(sweep_btn)
        vbox.addWidget(load_waveform_btn)
        vbox.addStretch()
        return vbox

    def _create_plot_canvas(self):
        self.canvas = FigureCanvas(Figure(figsize=(8, 5)))
        self.ax = self.canvas.figure.add_subplot(111)
        self.ax.set_title("Waveform Display")
        self.ax.set_xlabel("Sample Index")
        self.ax.set_ylabel("Amplitude")
        return self.canvas

    def _connect_awg(self):
        ip = self.ip_input.text().strip()
        self.awg.connect(ip)

    def _set_limits(self):
        try:
            self.awg.set_software_voltage_limit(float(self.voltage_input.text()))
            self.awg.set_software_power_limit(float(self.power_input.text()))
            self.awg.set_software_freq_limit(float(self.freq_input.text()))
        except ValueError:
            QMessageBox.warning(self, "Invalid Input", "Please enter numeric values.")

    def _set_user_parameters(self):
        try:
            success = self.awg.set_user_parameters(
                float(self.center_freq_input.text()),
                float(self.bandwidth_input.text()),
                float(self.freq_step_input.text()),
                float(self.start_power_input.text()),
                float(self.stop_power_input.text()),
                float(self.power_step_input.text())
            )
            if not success:
                QMessageBox.warning(self, "Parameter Error", "Some parameters exceed software limits.")
        except ValueError:
            QMessageBox.warning(self, "Invalid Input", "Please enter numeric values.")

    def _load_waveform(self):
        path, _ = QFileDialog.getOpenFileName(self, "Select Waveform File", "", "CSV or MAT Files (*.csv *.mat)")
        if path:
            threading.Thread(target=self.awg.load_waveform_file, args=(path,)).start()


# --- Run the GUI ---
if __name__ == "__main__":
    #from AWG_controller import AWG_controller  # Assuming your AWG controller code is in AWG_controller.py

    app = QApplication(sys.argv)
    awg = AWG_controller()
    gui = AWG_GUI_Interface(awg)
    gui.show()
    sys.exit(app.exec_())


QSocketNotifier: Can only be used with threads started with QThread


Connection error: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.
Software voltage limit set to 30.0 V
Software power limit set to 18.0 W
Software frequency limit set to 6000.0 MHz
User parameters set successfully.
Sweeping: Freq = 4750.00 to 5250.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 4850.00 to 5350.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 4950.00 to 5450.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 5050.00 to 5550.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 5150.00 to 5650.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 5250.00 to 5750.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 5350.00 to 5850.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0
Sweeping: Freq = 5450.00 to 5950.00 MHz
Power is 1.0
Power is 1.5
Power is 2.0


Exception in thread Thread-208 (sweep_center_band_power):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/student/venvs/vscode_jupyter_env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_11497/574194930.py", line 150, in sweep_center_band_power
AttributeError: 'NoneType' object has no attribute 'write'
